In [0]:
!rm -rf /content/Image-Classification

In [2]:
!git clone https://github.com/anthony0727/Image-Classification.git

Cloning into 'Image-Classification'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 351 (delta 32), reused 46 (delta 22), pack-reused 292
Receiving objects: 100% (351/351), 64.62 KiB | 4.31 MiB/s, done.
Resolving deltas: 100% (200/200), done.
/content/Image-Classification


In [1]:
%cd /content/Image-Classification

/content/Image-Classification


# Load Module

In [2]:
import tensorflow as tf
import gc

from model.vgg import VGG
from util import train

W0722 08:54:25.826234 140326408988544 deprecation_wrapper.py:119] From /content/Image-Classification/model/googlenet.py:9: The name tf.initializers.he_uniform is deprecated. Please use tf.compat.v1.initializers.he_uniform instead.

W0722 08:54:25.828428 140326408988544 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1288: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0722 08:54:25.833087 140326408988544 deprecation_wrapper.py:119] From /content/Image-Classification/model/vgg.py:14: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W0722 08:54:25.838398 140326408988544 deprecation_wrapper.py:119] From /content/Image-Classification/util/jpynb_helper.py:3: The name tf.GraphDef is deprecated. Please 

# Declare Static Variables

In [0]:
log_dir = './log'

In [0]:
input_shape = (32, 32, 3)
n_class = 100

# Build VGG11

In [5]:
pretrained_vgg = VGG(11)
pretrained_vgg.build(input_shape, n_class)

W0722 08:54:26.489790 140326408988544 deprecation_wrapper.py:119] From /content/Image-Classification/model/ABCNet.py:19: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0722 08:54:26.494073 140326408988544 deprecation_wrapper.py:119] From /content/Image-Classification/model/vgg.py:65: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 08:54:26.500527 140326408988544 deprecation_wrapper.py:119] From /content/Image-Classification/model/vgg.py:75: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0722 08:54:26.501930 140326408988544 deprecation_wrapper.py:119] From /content/Image-Classification/model/vgg.py:26: The name tf.layers.Conv2D is deprecated. Please use tf.compat.v1.layers.Conv2D instead.

W0722 08:54:26.742947 140326408988544 deprecation_wrapper.py:119] From /content/Image-Classification/model/vgg.py:41: The name tf.layers.MaxPooling2D

# Pretrain VGG11

In [6]:
with pretrained_vgg.graph.as_default() as graph:
    loss = graph.get_tensor_by_name('loss:0')
    lr = tf.placeholder_with_default(1e-2, (), name='learning_rate')
    global_step = tf.train.get_or_create_global_step()

    with tf.variable_scope('optimizer'):
        tf.train.MomentumOptimizer(lr, 0.9).minimize(loss, global_step)

    sess = tf.Session(graph=graph)
    sess = train(sess, log_dir, n_epoch=2)

Using TensorFlow backend.
W0722 08:54:30.011411 140326408988544 deprecation_wrapper.py:119] From /content/Image-Classification/util/train_helper.py:33: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

100%|██████████| 390/390 [00:29<00:00, 13.14it/s]


[  0 epoch train top-1 acc : 2.17% | top-5 acc : 9.80%


  1%|          | 2/390 [00:00<00:29, 13.18it/s]

[  0 epoch test top-1 acc : 1.70% | top-5 acc : 8.30%


100%|██████████| 390/390 [00:28<00:00, 13.69it/s]


[  1 epoch train top-1 acc : 2.06% | top-5 acc : 10.59%
[  1 epoch test top-1 acc : 1.20% | top-5 acc : 10.20%


In [0]:
with pretrained_vgg.graph.as_default() as graph:
    saver = tf.train.Saver()
    saver.save(sess, './vgg/vgg11')

# Reserve some memory

In [8]:
del pretrained_vgg.graph
gc.collect()

0

# Build VGG13

In [0]:
reconstructed_vgg = VGG(13)
reconstructed_vgg.build(input_shape, n_class)

# Collect weights to transfer

In [0]:
with reconstructed_vgg.graph.as_default():
    transfer_weights = \
        tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, '(VGGBLOCK-1/conv1)') + \
        tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, '(VGGBLOCK-2/conv1)') + \
        tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, '(VGGBLOCK-3/conv1|conv2)') + \
        tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'FC/')

# Restore pretrained weights

In [11]:
with reconstructed_vgg.graph.as_default() as graph:
    sess = tf.Session(graph=graph)
    
    saver = tf.train.Saver(var_list=transfer_weights)
    saver.restore(sess, './vgg/vgg11')

W0722 08:55:53.791559 140326408988544 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


# Declare necessaries and do initialization

In [0]:
def initialize_uninitialized(sess, vs):
    with sess.graph.as_default():
        is_not_initialized = sess.run([tf.is_variable_initialized(v) for v in vs])
        uninitialized_vs = [v for (v, f) in zip(vs, is_not_initialized) if not f]
        
        if len(uninitialized_vs):
            sess.run(tf.variables_initializer(uninitialized_vs))

In [0]:
with reconstructed_vgg.graph.as_default() as graph:
    loss = graph.get_tensor_by_name('loss:0')
    lr = tf.placeholder_with_default(1e-2, (), name='learning_rate')
    global_step = tf.train.get_or_create_global_step()

    with tf.variable_scope('optimizer'):
        tf.train.AdamOptimizer(lr).minimize(loss, global_step)

    with tf.variable_scope('initialization'):
        initialize_uninitialized(sess, tf.global_variables())
        initialize_uninitialized(sess, tf.local_variables()) 

# Check accuracy consistency

feed sample data to both vgg11 and vgg13 and check consistency

In [14]:
with reconstructed_vgg.graph.as_default() as graph:
    print(sess.run(graph.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)))
    print(sess.run(graph.get_collection(tf.GraphKeys.METRIC_VARIABLES)))

# Train VGG13

## initializer
    

## optimizer
    AdamOptimizer

In [16]:
with reconstructed_vgg.graph.as_default() as graph:
    
    sess = tf.Session(graph=graph)
    sess = train(sess, log_dir, n_epoch=2)
    saver = tf.train.Saver()
    saver.save(sess, './vgg/vgg11')

100%|██████████| 390/390 [00:37<00:00, 10.47it/s]


[  0 epoch train top-1 acc : 1.00% | top-5 acc : 5.00%


  0%|          | 1/390 [00:00<00:39,  9.89it/s]

[  0 epoch test top-1 acc : 1.50% | top-5 acc : 6.00%


100%|██████████| 390/390 [00:37<00:00, 10.47it/s]


[  1 epoch train top-1 acc : 1.00% | top-5 acc : 5.00%
[  1 epoch test top-1 acc : 1.10% | top-5 acc : 5.80%
